In [ ]:
import sys
sys.path.insert(0, "../../..")

import openghg

import glob
from pathlib import Path

%load_ext autoreload
%autoreload 2

First we want to create a list of files to process, we'll use some files from our test directory

In [ ]:
decc_files = glob.glob("../../../tests/data/proc_test_data/CRDS/*.dat")

In [ ]:
agage_files = glob.glob("../../../tests/data/proc_test_data/GC/*.C")

In [ ]:
decc_files

In [ ]:
agage_files